# Building A RAG System with Gemma, MongoDB and Open Source Models

Authored By: [Richmond Alake](https://huggingface.co/RichmondMongo)

In [1]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU
# !pip install accelerate

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 1.0 MB/s eta 0:00:00
  Using cached torch-2.3.1-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached safetensors-0.4.3-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB

In [1]:
# Load Dataset
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies
dataset = load_dataset("AIatMongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

c:\Users\jkkn7\PycharmProjects\adaptiveRAG-music-recommendation\aragvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,plot,genres,runtime,cast,num_mflix_comments,poster,title,fullplot,languages,directors,writers,awards,imdb,countries,type,plot_embedding,rated,metacritic
0,Young Pauline is left a lot of money when her ...,[Action],199.0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,Young Pauline is left a lot of money when her ...,[English],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],movie,"[0.0007293965299999999, -0.026834568000000003,...",None,NaN
1,A penniless young man tries to save an heiress...,"[Comedy, Short, Action]",22.0,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,As a penniless man worries about how he will m...,[English],"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 10146, 'rating': 7.0, 'votes': 639}",[USA],movie,"[-0.022837115, -0.022941574000000003, 0.014937...",TV-G,NaN
2,"Michael ""Beau"" Geste leaves England in disgrac...","[Action, Adventure, Drama]",101.0,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,None,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16634, 'rating': 6.9, 'votes': 222}",[USA],movie,"[0.00023330492999999998, -0.028511643000000003...",None,NaN
3,"Seeking revenge, an athletic young man joins t...","[Adventure, Action]",88.0,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,https://m.media-amazon.com/images/M/MV5BMzU0ND...,The Black Pirate,A nobleman vows to avenge the death of his fat...,None,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16654, 'rating': 7.2, 'votes': 1146}",[USA],movie,"[-0.005927917, -0.033394486, 0.0015323418, -0....",None,NaN
4,An irresponsible young millionaire changes his...,"[Action, Comedy, Romance]",58.0,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,For Heaven's Sake,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",[English],[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[USA],movie,"[-0.0059373598, -0.026604708, -0.0070914757000...",PASSED,NaN


In [2]:
dataset_df.shape

(1500, 18)

In [3]:
# Data Preparation

# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with an open source embedding model from Hugging Face
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)


Number of missing values in each column after removal:
plot                    0
genres                  0
runtime                14
cast                    1
num_mflix_comments      0
poster                 78
title                   0
fullplot                0
languages               1
directors              12
writers                13
awards                  0
imdb                    0
countries               0
type                    0
plot_embedding          1
rated                 279
metacritic            893
dtype: int64


,plot,genres,runtime,cast,num_mflix_comments,poster,title,fullplot,languages,directors,writers,awards,imdb,countries,type,rated,metacritic
0,Young Pauline is left a lot of money when her ...,[Action],199.0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,Young Pauline is left a lot of money when her ...,[English],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],movie,None,NaN
1,A penniless young man tries to save an heiress...,"[Comedy, Short, Action]",22.0,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,As a penniless man worries about how he will m...,[English],"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 10146, 'rating': 7.0, 'votes': 639}",[USA],movie,TV-G,NaN
2,"Michael ""Beau"" Geste leaves England in disgrac...","[Action, Adventure, Drama]",101.0,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,None,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16634, 'rating': 6.9, 'votes': 222}",[USA],movie,None,NaN
3,"Seeking revenge, an athletic young man joins t...","[Adventure, Action]",88.0,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,https://m.media-amazon.com/images/M/MV5BMzU0ND...,The Black Pirate,A nobleman vows to avenge the death of his fat...,None,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16654, 'rating': 7.2, 'votes': 1146}",[USA],movie,None,NaN
4,An irresponsible young millionaire changes his...,"[Action, Comedy, Romance]",58.0,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,For Heaven's Sake,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",[English],[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[USA],movie,PASSED,NaN


In [4]:
print(dataset_df.shape, dataset_df['fullplot'][0])

(1452, 17) Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take possession of her inheritance. Meanwhile, her "guardian" and his confederates constantly come up with schemes to get rid of Pauline so that he can get his hands on the money himself.


In [8]:
# dataset_df.to_csv("../data/movie-metadata.csv", index=False)

# Convert DataFrame to JSON and save to a file
file_path = '../data/movie-metadata.json'
dataset_df.to_json(file_path, orient='records', lines=True)